In [4]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [5]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display=Display(visible=0,size=(400,300))
    display.start()

In [6]:
# default_exp fastai.loop

In [7]:
# export
# Python native modules
import os,sys
from copy import deepcopy,copy
from typing import *
import types
import logging
import inspect
# Third party libs
from fastcore.all import *
import numpy as np
# Local modules

_logger=logging.getLogger(__name__)

# Loop
> fastrl concept of generic loop objects. 

The goal for Loops is to make it easy to customize, and know how sections of code connects
to other parts.

### Why do we need this?
We have identified at least 3 different kinds of loops already:

    Learner (training)
    Source/Gym (Data Access)
    Agent (How an AI takes in data, generates actions)

### What is a loop?

    It should be capable of containing inner loops. 
    It should self-describe its structure. 
    It should be easy to know which parts of the loop are taking long/short amounts of time.
    It should be flexible in state modification.
    It should alternatively make it easy show what fields are being changed at what points in time.

A Loop will act as a compiled structure. The actual result will be a compiled list of nodes that reference the original loop.

In [109]:
# export
EVENT_ORDER_MAPPING={}
PREFIXES=['before_','on_','after_','failed_','finally_']

def _is_event(o): return issubclass(o.__class__,Event)
def _last_element(ls): return ls[-1]

class NodeException(Exception):pass

class Event(object):
    def __init__(self,
                 function:Callable,
                ):
        store_attr()
        # We set the order over the entire Loop definition
        full_name=function.__module__+'.'+function.__qualname__.split('.')[0]
        if full_name not in EVENT_ORDER_MAPPING: self.order=1
        else: self.order=EVENT_ORDER_MAPPING[full_name]
        EVENT_ORDER_MAPPING[full_name]=self.order+1
            
        if self.name.startswith('_') or not any(self.name.startswith(pre) for pre in PREFIXES):
            raise NodeException(f'{self.name} needs to start with any {PREFIXES}')
        
    def __call__(self,*args,**kwargs): 
        
        return self.function(*args,**kwargs)
    def __lt__(self,o:'Event'): return self.order<o.order
    @property
    def name(self): return self.function.__name__
    @property
    def prefix(self): return self.name.split('_')[0]
    @property
    def postfix(self): return '_'.join(self.name.split('_')[1:])

event=Event

In [110]:
def on_test():pass

decorated_on_test=Event(on_test)

In [111]:
class B():
    def on_test_2_a(self):pass

In [112]:
class A():
    @classmethod
    def events(cls):
        return L(inspect.getmembers(cls)).map(_last_element).filter(_is_event)

    @event
    def before_test_c(self):pass
    @event
    def on_test_c(self):pass
    @event
    def after_test_c(self):pass
    @event
    def after_test_c(self):pass
    @event
    def on_test_b(self):pass
    @event
    def on_test_a(self):pass

Notes:
- Having a section object might make this overal loop management better
    - remaining stuff, the Section object needs to handle lists and execute those also.
    - with this in mind, using the section I think will make the looping run much cleaner.

In [136]:
# export
def _grab_postfix(o:Event): return o.postfix
def _grab_prefix(o:Event): return o.prefix
def _default_raise(ex): raise 

class Section(object):
    
    def __init__(self,events):
        self.events={k:noop if k!='failed' else _default_raise for k in PREFIXES}
        self.events=merge(self.events,L(events).map(_grab_prefix))
        
    
    def __repr__(self): return str(self.__class__.__name__)+f' {len(self)} events'
    def __len__(self): return len(self.events) 

    def run(self):
        try:
            self.events['before_']()
            self.events['on_']()
            self.events['after_']()
        except ex:
            self.events['failed_']()
        finally:
            self.events['finally_']()
    
    @classmethod
    def from_events(cls,events:List[Event]):
        event_groups=groupby(events,_grab_postfix)
        return [cls(o) for o in event_groups.values()]

In [133]:
Section.from_events(A.events())[0].events

(#3) ['after','before','on']

In [101]:
# export
def _is_event(o): return issubclass(o.__class__,Event)
def _last_element(ls): return ls[-1]

class Loop(object):  
    
    @classmethod
    def events(cls,instance=None,instantiate=False):
        loop=ifnone(instance,cls()) if instantiate else cls
        events=L(inspect.getmembers(loop)).map(_last_element).filter(_is_event)
        
        
        
        return events

class Outer(Loop):

    
    @event
    def before_step(self) :  print('before_step')
    @event
    def on_step(self)     :  print('on_step')
    @event
    def after_step(self)  :  print('after_step')
    @event
    def failed_step(self) :  print('failed_step')
    @event
    def finally_step(self):  print('finally_step')
 
    @event
    def before_jump(self) :  print('before_jump')
    @event
    def on_jump(self)     :  print('on_jump')
    @event
    def after_jump(self)  :  print('after_jump')
    @event
    def failed_jump(self) :  print('failed_jump')
    @event
    def finally_jump(self):  print('finally_jump')

list(Outer.events())

In [95]:

class Inner(Loop):
    call_on=L(Outer.on_step,Outer.after_step,Outer.finally_jump)
    
    def run(self,nodes=None):
        print(self.tab,'--- ENTERING INNER LOOP ---')
        super().run(nodes)
        print(self.tab,'--- EXITING INNER LOOP ---')
    
    @event
    def before_iteration(self) : print(self.tab,'before_iteration')
    @event
    def on_iteration(self)     : print(self.tab,'on_iteration')
    @event
    def after_iteration(self)  : print(self.tab,'after_iteration')
    @event
    def failed_iteration(self) : print(self.tab,'failed_iteration')
    @event
    def finally_iteration(self): print(self.tab,'finally_iteration')
    
class FailingInner(Loop):
    call_on=L(Inner.finally_iteration)
    
    def run(self,nodes=None):
        print(self.tab,'--- ENTERING FailingInner LOOP ---')
        super().run(nodes)
        print(self.tab,'--- EXITING FailingInner LOOP ---')
    
    @event
    def on_force_fail(self):                    
        print(self.tab,'on_force_fail')
        raise Exception
        

In [79]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 02_fastai.loop.ipynb.
Converted 02_fastai.loop.old.ipynb.
Converted 02_fastai.loop.old.old.ipynb.
Converted 03_callback.core.ipynb.
Converted 04_agent.ipynb.
Converted 05_data.test_async.ipynb.
Converted 05a_data.block.ipynb.
Converted 05b_data.gym.ipynb.
Converted 06a_memory.experience_replay.ipynb.
Converted 06f_memory.tensorboard.ipynb.
Converted 10a_agents.dqn.core.ipynb.
Converted 10b_agents.dqn.targets.ipynb.
Converted 10c_agents.dqn.double.ipynb.
Converted 10d_agents.dqn.dueling.ipynb.
Converted 10e_agents.dqn.categorical.ipynb.
Converted 11a_agents.policy_gradient.ppo.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/02_fastai.loop.ipynb
